# Лабораторная работа №1 — алгоритм k ближайших соседей (KNN)

### Регрессия

#### Датасет

Для задачи регрессии был использован датасет [**Salary Data**](https://www.kaggle.com/datasets/rkiattisak/salaly-prediction-for-beginer)

Этот набор данных подходит для исследования факторов, влияющих на размер зарплаты. Такую модель можно применять, например, при анализе рынка труда или для предварительной оценки зарплат кандидатов в компаниях

#### Метрики

Для оценки качества регрессионной модели выбраны три наиболее используемые показателя:

1. **MAE** — средняя ошибка в исходных единицах
2. **MSE** — квадрат средних отклонений, чувствителен к выбросам
3. **R²** — показывает, насколько хорошо модель объясняет вариацию целевой переменной

---

### Классификация

#### Датасет

Для классификационной части лабораторной работы был выбран датасет [**Student Depression Dataset**](https://www.kaggle.com/datasets/hopesb/student-depression-dataset)

Он содержит данные о студентах и различных факторах, которые могут влиять на уровень депрессии. Использование такого набора позволяет построить модель для раннего выявления риска депрессии и анализа того, какие факторы оказывают наибольшее влияние. Это может быть полезно образовательным учреждениям при разработке программ поддержки студентов

#### Метрики

Для анализа работы классификатора используются следующие показатели:

1. **Accuracy** — общая доля правильных предсказаний
2. **Precision** — насколько точны предсказания конкретного класса
3. **Recall** — доля найденных объектов класса среди всех реальных объектов
4. **F1-score** — взвешенное гармоническое среднее между Precision и Recall


---

# Подключение библиотек и установка зависимостей

In [6]:
!pip install numpy pandas matplotlib scikit-learn kagglehub

# Импорт всех необходимых модулей

In [7]:
import os
import shutil
import kagglehub

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

# Cкачивание датасетов и сохранение в папку data

In [8]:
os.makedirs("data", exist_ok=True)

print("Скачиваю датасет Student Depression Dataset")
path_dep = kagglehub.dataset_download("hopesb/student-depression-dataset")
print("Файлы сохранены здесь:", path_dep)

print()
print("Cкачиваю датасет Salary Prediction dataset")
path_house = kagglehub.dataset_download("rkiattisak/salaly-prediction-for-beginer")
print("Файлы сохранены здесь:", path_house)

for src_dir in [path_dep, path_house]:
    for f in os.listdir(src_dir):
        full_src = os.path.join(src_dir, f)
        full_dst = os.path.join("data", f)
        if os.path.isfile(full_src):
            shutil.copy(full_src, full_dst)

print()
print("Датасеты успешно скопированы в папку ./data/")
print("Содержимое data:", os.listdir("data"))


Скачиваю датасет Student Depression Dataset
Файлы сохранены здесь: /root/.cache/kagglehub/datasets/hopesb/student-depression-dataset/versions/1

Cкачиваю датасет Salary Prediction dataset
Файлы сохранены здесь: /root/.cache/kagglehub/datasets/rkiattisak/salaly-prediction-for-beginer/versions/1

Датасеты успешно скопированы в папку ./data/
Содержимое data: ['Student Depression Dataset.csv', 'Salary Data.csv']


# Часть 1. Регрессия
Данные о зарплатах загружаются в DataFrame, и выводятся первые строки, чтобы убедиться, что файл прочитан корректно.

In [9]:
salary_df = pd.read_csv("data/Salary Data.csv")
print("Размер датасета Salary_Data:", salary_df.shape)

salary_df.head()

Размер датасета Salary_Data: (375, 6)


,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


Отобразим типы данных, количество NaN и основные статистики - это помогает понять структуру набора данных и подготовить его к обработке.

In [10]:
salary_df.info()

print("Основные статистики:")
salary_df.describe(include="all").T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  373 non-null    float64
 1   Gender               373 non-null    object 
 2   Education Level      373 non-null    object 
 3   Job Title            373 non-null    object 
 4   Years of Experience  373 non-null    float64
 5   Salary               373 non-null    float64
dtypes: float64(3), object(3)
memory usage: 17.7+ KB
Основные статистики:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,373.0,NaN,NaN,NaN,37.431635,7.069073,23.0,31.0,36.0,44.0,53.0
Gender,373,2,Male,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Education Level,373,3,Bachelor's,224,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Job Title,373,174,Director of Marketing,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of Experience,373.0,NaN,NaN,NaN,10.030831,6.557007,0.0,4.0,9.0,15.0,25.0
Salary,373.0,NaN,NaN,NaN,100577.345845,48240.013482,350.0,55000.0,95000.0,140000.0,250000.0


# Подготовка данных для регрессии

Удаляется столбец `Job Title`, так как он содержит слишком много уникальных значений и усложняет модель. Затем данные разделяются на обучающую и тестовую части. Это создает основу для построения первой модели

Категориальные признаки преобразуются в числовой формат с помощью OrdinalEncoder

Все пропуски заполняются наиболее распространёнными значениями. Это предотвращает ошибки во время обучения модели и упрощает работу с данными. Данный шаг важен, так как KNN не умеет работать с NaN


In [11]:
reg_df = salary_df.copy()
reg_df = reg_df.dropna(subset=["Salary"])

if "Job Title" in reg_df.columns:
    reg_df = reg_df.drop(columns=["Job Title"])

X_reg = reg_df.drop(columns=["Salary"])
y_reg = reg_df["Salary"]

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

reg_categorical_cols = ["Gender", "Education Level"]
reg_numeric_cols = [col for col in X_reg.columns if col not in reg_categorical_cols]

reg_ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

X_reg_train_base = X_reg_train.copy()
X_reg_test_base = X_reg_test.copy()

X_reg_train_base[reg_categorical_cols] = reg_ordinal_encoder.fit_transform(
    X_reg_train_base[reg_categorical_cols]
)
X_reg_test_base[reg_categorical_cols] = reg_ordinal_encoder.transform(
    X_reg_test_base[reg_categorical_cols]
)

reg_imputer = SimpleImputer(strategy="most_frequent")

X_reg_train_base = pd.DataFrame(
    reg_imputer.fit_transform(X_reg_train_base),
    columns=X_reg_train_base.columns
)
X_reg_test_base = pd.DataFrame(
    reg_imputer.transform(X_reg_test_base),
    columns=X_reg_test_base.columns
)

print("NaN в X_train_base:", X_reg_train_base.isna().sum().sum())
print("NaN в X_test_base:", X_reg_test_base.isna().sum().sum())


NaN в X_train_base: 0
NaN в X_test_base: 0


In [12]:
def regression_metrics(y_true, y_pred) -> dict:
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "R2": r2_score(y_true, y_pred),
    }


def print_regression_metrics(name: str, metrics: dict):
    print(f"\n{name}")
    print("-" * len(name))
    print(f"MSE: {metrics['MSE']:.2f}")
    print(f"MAE: {metrics['MAE']:.2f}")
    print(f"R^2: {metrics['R2']:.3f}")


def simple_regression_cv(model_cls, X, y, n_splits=5, **model_kwargs):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mse_list, mae_list, r2_list = [], [], []

    X = np.asarray(X)
    y = np.asarray(y)

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]

        model = model_cls(**model_kwargs)
        model.fit(X_tr, y_tr)
        y_val_pred = model.predict(X_val)

        mse_list.append(mean_squared_error(y_val, y_val_pred))
        mae_list.append(mean_absolute_error(y_val, y_val_pred))
        r2_list.append(r2_score(y_val, y_val_pred))

    cv_results = {
        "MSE_mean": np.mean(mse_list),
        "MAE_mean": np.mean(mae_list),
        "R2_mean": np.mean(r2_list),
    }
    return cv_results

# Бейзлайн для регрессии 

Создаётся и обучается простая модель KNN-регрессора с количеством соседей k=3. Мы выполняем кросс-валидацию и затем тестируем модель на отложенной выборке. Эта модель служит отправной точкой для дальнейших улучшений


In [13]:
reg_baseline_cv = simple_regression_cv(
    KNeighborsRegressor,
    X_reg_train_base.to_numpy(),
    y_reg_train.to_numpy(),
    n_splits=5,
    n_neighbors=3,
)

print("=== Регрессия: бейзлайн (train CV) ===")
for k, v in reg_baseline_cv.items():
    print(f"{k}: {v:.2f}")

reg_baseline_model = KNeighborsRegressor(n_neighbors=3)
reg_baseline_model.fit(X_reg_train_base, y_reg_train)

y_reg_pred_base = reg_baseline_model.predict(X_reg_test_base)
reg_base_test_metrics = regression_metrics(y_reg_test, y_reg_pred_base)

print_regression_metrics("Регрессия - бейзлайн (test)", reg_base_test_metrics)

=== Регрессия: бейзлайн (train CV) ===
MSE_mean: 312457775.25
MAE_mean: 11712.62
R2_mean: 0.86

Регрессия - бейзлайн (test)
---------------------------
MSE: 231615737.04
MAE: 10754.00
R^2: 0.903


# Улучшенный бейзлайн для регрессии

Сформулируем несколько гипотез по улучшению качества:

1. Заменить порядковую кодировку категориальных признаков на `OneHotEncoder`.
2. Отмасштабировать числовые признаки (`Age`, `Years of Experience`) с помощью `StandardScaler`.
3. Подобрать число соседей `k` по результатам кросс-валидации.

Дальше последовательно реализуем эти шаги.


In [14]:
reg_onehot = OneHotEncoder(
    drop="first",
    sparse_output=False,
    handle_unknown="ignore",
)

reg_cat_train_ohe = reg_onehot.fit_transform(X_reg_train[reg_categorical_cols])
reg_cat_test_ohe = reg_onehot.transform(X_reg_test[reg_categorical_cols])

reg_ohe_cols = reg_onehot.get_feature_names_out(reg_categorical_cols)

X_reg_train_ohe = pd.concat(
    [
        X_reg_train[reg_numeric_cols].reset_index(drop=True),
        pd.DataFrame(reg_cat_train_ohe, columns=reg_ohe_cols),
    ],
    axis=1,
)
X_reg_test_ohe = pd.concat(
    [
        X_reg_test[reg_numeric_cols].reset_index(drop=True),
        pd.DataFrame(reg_cat_test_ohe, columns=reg_ohe_cols),
    ],
    axis=1,
)

reg_scaler = StandardScaler()

X_reg_train_scaled = X_reg_train_ohe.copy()
X_reg_test_scaled = X_reg_test_ohe.copy()

X_reg_train_scaled[reg_numeric_cols] = reg_scaler.fit_transform(
    X_reg_train_ohe[reg_numeric_cols]
)
X_reg_test_scaled[reg_numeric_cols] = reg_scaler.transform(
    X_reg_test_ohe[reg_numeric_cols]
)

X_reg_train_scaled.head()

,Age,Years of Experience,Gender_Male,Education Level_Master's,Education Level_PhD
0,-0.496349,-0.469123,1.0,0.0,0.0
1,-0.069680,-0.007995,1.0,0.0,0.0
2,-1.207465,-1.237669,0.0,0.0,0.0
3,-0.638572,-0.776541,1.0,0.0,0.0
4,-0.638572,-0.469123,0.0,1.0,0.0


In [15]:
candidate_k = [i for i in range(1,10)]
cv_results_reg = []

for k_val in candidate_k:
    cv_res = simple_regression_cv(
        KNeighborsRegressor,
        X_reg_train_scaled.to_numpy(),
        y_reg_train.to_numpy(),
        n_splits=5,
        n_neighbors=k_val,
    )
    cv_results_reg.append((k_val, cv_res))
    print(f"k = {k_val}: R2_mean = {cv_res['R2_mean']:.3f}, MSE_mean = {cv_res['MSE_mean']:.2f}")

print()
best_k_reg = max(cv_results_reg, key=lambda x: x[1]["R2_mean"])[0]
print("Лучшее k по CV для регрессии:", best_k_reg)

reg_improved_model = KNeighborsRegressor(n_neighbors=best_k_reg)
reg_improved_model.fit(X_reg_train_scaled, y_reg_train)

y_reg_pred_improved = reg_improved_model.predict(X_reg_test_scaled)
reg_improved_test_metrics = regression_metrics(y_reg_test, y_reg_pred_improved)

print_regression_metrics("Регрессия - улучшенный бейзлайн (test)", reg_improved_test_metrics)

k = 1: R2_mean = 0.831, MSE_mean = 380374507.20
k = 2: R2_mean = 0.879, MSE_mean = 276624095.48
k = 3: R2_mean = 0.878, MSE_mean = 280862278.39
k = 4: R2_mean = 0.884, MSE_mean = 265322987.43
k = 5: R2_mean = 0.898, MSE_mean = 235271265.73
k = 6: R2_mean = 0.900, MSE_mean = 230937375.38
k = 7: R2_mean = 0.903, MSE_mean = 223147277.64
k = 8: R2_mean = 0.904, MSE_mean = 220971095.07
k = 9: R2_mean = 0.901, MSE_mean = 227458107.25

Лучшее k по CV для регрессии: 8

Регрессия - улучшенный бейзлайн (test)
--------------------------------------
MSE: 240613176.04
MAE: 9407.17
R^2: 0.900


# Собственная реализация KNN для регрессии

Теперь реализуем свой класс `MyKNNRegressor`:

- алгоритм: классический KNN;
- расстояние: евклидово по умолчанию;
- предсказание: среднее значение целевой переменной ближайших соседей.

Дальше:

1. Обучим свою модель на тех же данных, что и бейзлайн.
2. Сравним качество со `sklearn`-версией.
3. Применим к своей модели те же улучшения (one-hot + масштабирование + оптимальное k) и сравним с улучшенным бейзлайном.


In [16]:
class MyKNNRegressor:
    def __init__(self, n_neighbors=5, metric="euclidean"):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = np.asarray(X, dtype=float)
        self.y_train = np.asarray(y, dtype=float)

    def _compute_distances(self, x):
        diff = self.X_train - x
        if self.metric == "euclidean":
            dists = np.sqrt((diff * diff).sum(axis=1))
        elif self.metric == "manhattan":
            dists = np.abs(diff).sum(axis=1)
        else:
            raise ValueError(f"Неизвестная метрика: {self.metric}")
        return dists

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        preds = []

        for x in X:
            dists = self._compute_distances(x)
            
            k = self.n_neighbors
            idx = np.argpartition(dists, k)[:k]

            neighbor_vals = self.y_train[idx]
            preds.append(neighbor_vals.mean())

        return np.array(preds)

In [17]:
reg_cv_my_base = simple_regression_cv(
    MyKNNRegressor,
    X_reg_train_base.to_numpy(),
    y_reg_train.to_numpy(),
    n_splits=5,
    n_neighbors=3,
)

print("=== Регрессия: собственная реализация (train CV, без улучшений) ===")
for k, v in reg_cv_my_base.items():
    print(f"{k}: {v:.2f}")

my_regressor_base = MyKNNRegressor(n_neighbors=3)
my_regressor_base.fit(X_reg_train_base, y_reg_train)

y_reg_pred_my_base = my_regressor_base.predict(X_reg_test_base)
reg_my_base_test_metrics = regression_metrics(y_reg_test, y_reg_pred_my_base)

print_regression_metrics("Регрессия - собственная реализация (test, без улучшений)", reg_my_base_test_metrics)


=== Регрессия: собственная реализация (train CV, без улучшений) ===
MSE_mean: 278790323.90
MAE_mean: 10841.15
R2_mean: 0.88

Регрессия - собственная реализация (test, без улучшений)
--------------------------------------------------------
MSE: 240837959.26
MAE: 10642.89
R^2: 0.900


In [18]:
reg_cv_my_improved = simple_regression_cv(
    MyKNNRegressor,
    X_reg_train_scaled.to_numpy(),
    y_reg_train.to_numpy(),
    n_splits=5,
    n_neighbors=best_k_reg,
)

print("=== Регрессия: собственная реализация (train CV, с улучшениями) ===")
for k, v in reg_cv_my_improved.items():
    print(f"{k}: {v:.2f}")

# Обучение и оценка на test
my_regressor_improved = MyKNNRegressor(n_neighbors=best_k_reg)
my_regressor_improved.fit(X_reg_train_scaled, y_reg_train)

y_reg_pred_my_improved = my_regressor_improved.predict(X_reg_test_scaled)
reg_my_improved_test_metrics = regression_metrics(y_reg_test, y_reg_pred_my_improved)

print_regression_metrics("Регрессия — собственная реализация (test, с улучшениями)", reg_my_improved_test_metrics)

=== Регрессия: собственная реализация (train CV, с улучшениями) ===
MSE_mean: 219369620.84
MAE_mean: 9999.55
R2_mean: 0.90

Регрессия — собственная реализация (test, с улучшениями)
--------------------------------------------------------
MSE: 241488905.21
MAE: 9373.83
R^2: 0.899


## Часть 2. Классификация

В этой ячейке загружается датасет, используемый для решения задачи классификации - определения депрессии у студентов. Мы выводим размер таблицы и первые строки, чтобы убедиться, что данные корректно считались и имеют ожидаемую структуру

In [19]:
stud_df = pd.read_csv("data/Student Depression Dataset.csv")
print("Размер датасета Student_Depression_Dataset:", stud_df.shape)

stud_df.head()

Размер датасета Student_Depression_Dataset: (27901, 18)


,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


В этой части кода создаётся копия исходного датасета, после чего формируются матрица признаков и целевая переменная для задачи классификации. Далее данные разделяются на обучающую и тестовую выборки с использованием стратификации - это позволяет сохранить исходное распределение классов и избежать перекоса при обучении. Затем задаются списки категориальных и численных признаков, что важно для последующей обработки данных. Пропуски в датасете заполняются наиболее частыми значениями с помощью SimpleImputer, после чего формируется обновлённая обучающая таблица. Такой этап подготовки необходим, чтобы модель корректно работала и не сталкивалась с отсутствующими значениями

In [20]:
clf_df = stud_df.copy()

X_clf = clf_df.drop(columns=["Depression", "id"])
y_clf = clf_df["Depression"]

X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf,
    y_clf,
    test_size=0.2,
    random_state=42,
    stratify=y_clf,
)

clf_categorical_cols = [
    "Gender",
    "City",
    "Profession",
    "Sleep Duration",
    "Dietary Habits",
    "Degree",
    "Have you ever had suicidal thoughts ?",
    "Family History of Mental Illness",
]

clf_numeric_cols = [
    "Age",
    "Academic Pressure",
    "Work Pressure",
    "CGPA",
    "Study Satisfaction",
    "Job Satisfaction",
    "Work/Study Hours",
    "Financial Stress",
]

clf_imputer = SimpleImputer(strategy="most_frequent")

X_clf_train_imp = pd.DataFrame(
    clf_imputer.fit_transform(X_clf_train),
    columns=X_clf_train.columns,
)
X_clf_test_imp = pd.DataFrame(
    clf_imputer.transform(X_clf_test),
    columns=X_clf_test.columns,
)

X_clf_train_imp.head()

,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,Male,18.0,Jaipur,Student,4.0,0.0,6.02,1.0,0.0,7-8 hours,Moderate,Class 12,Yes,3.0,5.0,No
1,Male,25.0,Vadodara,Student,3.0,0.0,6.37,2.0,0.0,7-8 hours,Moderate,B.Arch,No,9.0,1.0,Yes
2,Male,30.0,Ahmedabad,Student,3.0,0.0,9.24,2.0,0.0,7-8 hours,Unhealthy,M.Ed,Yes,5.0,5.0,Yes
3,Male,34.0,Bhopal,Student,3.0,0.0,7.37,5.0,0.0,7-8 hours,Moderate,B.Com,Yes,12.0,3.0,No
4,Male,25.0,Patna,Student,3.0,0.0,7.47,4.0,0.0,5-6 hours,Unhealthy,B.Com,No,11.0,5.0,No


В этой ячейке определяются функции, необходимые для оценки качества моделей классификации. Сначала реализованы метрики - точность, полнота, точность предсказания и F1-мера, которые помогают количественно описать качество модели при разных типах ошибок. Далее создаётся удобная функция вывода результатов, чтобы отображать метрики в структурированном виде. Завершает блок функция кросс-валидации - она многократно разбивает выборку на обучающую и проверочную части, обучает модель и собирает усреднённые метрики

In [21]:
def classification_metrics(y_true, y_pred) -> dict:
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average="weighted", zero_division=0),
        "Recall": recall_score(y_true, y_pred, average="weighted", zero_division=0),
        "F1": f1_score(y_true, y_pred, average="weighted", zero_division=0),
    }


def print_classification_metrics(name: str, metrics: dict):
    print(f"\n{name}")
    print("-" * len(name))
    print(f"Accuracy:  {metrics['Accuracy']:.3f}")
    print(f"Precision: {metrics['Precision']:.3f}")
    print(f"Recall:    {metrics['Recall']:.3f}")
    print(f"F1-score:  {metrics['F1']:.3f}")


def simple_classification_cv(model_cls, X, y, n_splits=5, **model_kwargs):

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    acc_list, prec_list, rec_list, f1_list = [], [], [], []

    X = np.asarray(X)
    y = np.asarray(y)

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]

        model = model_cls(**model_kwargs)
        model.fit(X_tr, y_tr)
        y_val_pred = model.predict(X_val)

        m = classification_metrics(y_val, y_val_pred)
        acc_list.append(m["Accuracy"])
        prec_list.append(m["Precision"])
        rec_list.append(m["Recall"])
        f1_list.append(m["F1"])

    cv_results = {
        "Accuracy_mean": np.mean(acc_list),
        "Precision_mean": np.mean(prec_list),
        "Recall_mean": np.mean(rec_list),
        "F1_mean": np.mean(f1_list),
    }
    return cv_results

# Бейзлайн для классификации (KNeighborsClassifier)

Здесь выполняется базовая подготовка данных для классификации: категориальные признаки кодируются через OrdinalEncoder, чтобы модель могла работать с ними как с числами. Затем проводится кросс-валидация модели KNN с k=3, после чего модель обучается на тренировочных данных и оценивается на тестовых, формируя бейзлайн-качество для дальнейших сравнений.


In [22]:
clf_ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

X_clf_train_base = X_clf_train_imp.copy()
X_clf_test_base = X_clf_test_imp.copy()

X_clf_train_base[clf_categorical_cols] = clf_ordinal_encoder.fit_transform(
    X_clf_train_base[clf_categorical_cols]
)
X_clf_test_base[clf_categorical_cols] = clf_ordinal_encoder.transform(
    X_clf_test_base[clf_categorical_cols]
)

clf_baseline_cv = simple_classification_cv(
    KNeighborsClassifier,
    X_clf_train_base.to_numpy(),
    y_clf_train.to_numpy(),
    n_splits=5,
    n_neighbors=3,
)

print("=== Классификация: бейзлайн (train CV) ===")
for k, v in clf_baseline_cv.items():
    print(f"{k}: {v:.3f}")

clf_baseline_model = KNeighborsClassifier(n_neighbors=3)
clf_baseline_model.fit(X_clf_train_base, y_clf_train)

y_clf_pred_base = clf_baseline_model.predict(X_clf_test_base)
clf_base_test_metrics = classification_metrics(y_clf_test, y_clf_pred_base)

print_classification_metrics("Классификация - бейзлайн (test)", clf_base_test_metrics)

=== Классификация: бейзлайн (train CV) ===
Accuracy_mean: 0.733
Precision_mean: 0.731
Recall_mean: 0.733
F1_mean: 0.730

Классификация - бейзлайн (test)
-------------------------------
Accuracy:  0.739
Precision: 0.737
Recall:    0.739
F1-score:  0.737


# Улучшенный бейзлайн для классификации

В этой ячейке проводится расширенная обработка данных: категориальные признаки кодируются с помощью OneHotEncoder, а численные - масштабируются через StandardScaler, что помогает улучшить работу KNN. Затем подбирается оптимальное число соседей k методом кросс-валидации, после чего обучается улучшенная модель и оценивается качество её работы на тестовых данных

In [23]:
clf_onehot = OneHotEncoder(
    drop="first",
    sparse_output=False,
    handle_unknown="ignore",
)

clf_cat_train_ohe = clf_onehot.fit_transform(X_clf_train_imp[clf_categorical_cols])
clf_cat_test_ohe = clf_onehot.transform(X_clf_test_imp[clf_categorical_cols])

clf_ohe_cols = clf_onehot.get_feature_names_out(clf_categorical_cols)

X_clf_train_ohe = pd.concat(
    [
        X_clf_train_imp[clf_numeric_cols].reset_index(drop=True),
        pd.DataFrame(clf_cat_train_ohe, columns=clf_ohe_cols),
    ],
    axis=1,
)
X_clf_test_ohe = pd.concat(
    [
        X_clf_test_imp[clf_numeric_cols].reset_index(drop=True),
        pd.DataFrame(clf_cat_test_ohe, columns=clf_ohe_cols),
    ],
    axis=1,
)

clf_scaler = StandardScaler()

X_clf_train_scaled = X_clf_train_ohe.copy()
X_clf_test_scaled = X_clf_test_ohe.copy()

X_clf_train_scaled[clf_numeric_cols] = clf_scaler.fit_transform(
    X_clf_train_ohe[clf_numeric_cols]
)
X_clf_test_scaled[clf_numeric_cols] = clf_scaler.transform(
    X_clf_test_ohe[clf_numeric_cols]
)

candidate_k_clf = [3, 5, 7, 9]
cv_results_clf = []

for k_val in candidate_k_clf:
    cv_res = simple_classification_cv(
        KNeighborsClassifier,
        X_clf_train_scaled.to_numpy(),
        y_clf_train.to_numpy(),
        n_splits=5,
        n_neighbors=k_val,
    )
    cv_results_clf.append((k_val, cv_res))
    print(
        f"k = {k_val}: F1_mean = {cv_res['F1_mean']:.3f}, "
        f"Accuracy_mean = {cv_res['Accuracy_mean']:.3f}"
    )

best_k_clf = max(cv_results_clf, key=lambda x: x[1]["F1_mean"])[0]
print("\nЛучшее k по CV для классификации:", best_k_clf)

clf_improved_model = KNeighborsClassifier(n_neighbors=best_k_clf)
clf_improved_model.fit(X_clf_train_scaled, y_clf_train)

y_clf_pred_improved = clf_improved_model.predict(X_clf_test_scaled)
clf_improved_test_metrics = classification_metrics(y_clf_test, y_clf_pred_improved)

print_classification_metrics("Классификация - улучшенный бейзлайн (test)", clf_improved_test_metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


k = 3: F1_mean = 0.797, Accuracy_mean = 0.799
k = 5: F1_mean = 0.812, Accuracy_mean = 0.814
k = 7: F1_mean = 0.817, Accuracy_mean = 0.819
k = 9: F1_mean = 0.821, Accuracy_mean = 0.823

Лучшее k по CV для классификации: 9

Классификация - улучшенный бейзлайн (test)
------------------------------------------
Accuracy:  0.825
Precision: 0.825
Recall:    0.825
F1-score:  0.823


# Собственная реализация KNN для классификации

В этой ячейке реализуется собственный класс MyKNNClassifier - упрощённая версия алгоритма KNN без использования sklearn. В методе fit сохраняются обучающие данные, а при predict для каждого объекта вычисляются расстояния до всех элементов обучающей выборки и определяется наиболее частый класс среди k ближайших соседей


In [24]:
from collections import Counter

class MyKNNClassifier:
    def __init__(self, n_neighbors=5, metric="euclidean"):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = np.asarray(X, dtype=float)
        self.y_train = np.asarray(y)

    def _compute_distances(self, x):
        diff = self.X_train - x
        if self.metric == "euclidean":
            dists = np.sqrt((diff * diff).sum(axis=1))
        elif self.metric == "manhattan":
            dists = np.abs(diff).sum(axis=1)
        else:
            raise ValueError(f"Неизвестная метрика: {self.metric}")
        return dists

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        preds = []

        for x in X:
            dists = self._compute_distances(x)

            k = self.n_neighbors
            idx = np.argpartition(dists, k)[:k]
            neighbor_labels = self.y_train[idx]

            counter = Counter(neighbor_labels)
            most_common_label = counter.most_common(1)[0][0]
            preds.append(most_common_label)

        return np.array(preds)

В этой ячейке проводится оценка базовой версии собственного классификатора MyKNNClassifier: сначала выполняется кросс-валидация, чтобы получить средние метрики на обучающей выборке, а затем модель обучается на train-данных и проверяется на test-наборе без каких-либо улучшени

In [25]:
clf_cv_my_base = simple_classification_cv(
    MyKNNClassifier,
    X_clf_train_base.to_numpy(),
    y_clf_train.to_numpy(),
    n_splits=5,
    n_neighbors=3,
)

print("=== Классификация: собственная реализация (train CV, без улучшений) ===")
for k, v in clf_cv_my_base.items():
    print(f"{k}: {v:.3f}")

my_clf_base = MyKNNClassifier(n_neighbors=3)
my_clf_base.fit(X_clf_train_base, y_clf_train)

y_clf_pred_my_base = my_clf_base.predict(X_clf_test_base)
clf_my_base_test_metrics = classification_metrics(y_clf_test, y_clf_pred_my_base)

print_classification_metrics("Классификация - собственная реализация (test, без улучшений)", clf_my_base_test_metrics)

=== Классификация: собственная реализация (train CV, без улучшений) ===
Accuracy_mean: 0.733
Precision_mean: 0.731
Recall_mean: 0.733
F1_mean: 0.730

Классификация - собственная реализация (test, без улучшений)
------------------------------------------------------------
Accuracy:  0.739
Precision: 0.737
Recall:    0.739
F1-score:  0.737


В этой ячейке оценивается улучшенная версия собственного KNN-классификатора: используется масштабированная обучающая выборка и оптимальное число соседей, найденное ранее. Сначала выполняется кросс-валидация на улучшенных признаках, затем модель обучается и тестируется

In [26]:
clf_cv_my_improved = simple_classification_cv(
    MyKNNClassifier,
    X_clf_train_scaled.to_numpy(),
    y_clf_train.to_numpy(),
    n_splits=5,
    n_neighbors=best_k_clf,
)

print("=== Классификация: собственная реализация (train CV, с улучшениями) ===")
for k, v in clf_cv_my_improved.items():
    print(f"{k}: {v:.3f}")

my_clf_improved = MyKNNClassifier(n_neighbors=best_k_clf)
my_clf_improved.fit(X_clf_train_scaled, y_clf_train)

y_clf_pred_my_improved = my_clf_improved.predict(X_clf_test_scaled)
clf_my_improved_test_metrics = classification_metrics(y_clf_test, y_clf_pred_my_improved)

print_classification_metrics("Классификация - собственная реализация (test, с улучшениями)", clf_my_improved_test_metrics)

=== Классификация: собственная реализация (train CV, с улучшениями) ===
Accuracy_mean: 0.823
Precision_mean: 0.823
Recall_mean: 0.823
F1_mean: 0.821

Классификация - собственная реализация (test, с улучшениями)
------------------------------------------------------------
Accuracy:  0.825
Precision: 0.825
Recall:    0.825
F1-score:  0.823


## Итоги

Модели KNN для регрессии и классификации показали, что качество существенно зависит от предварительной обработки данных и корректного подбора числа соседей. Улучшенные варианты с нормализацией и расширенным кодированием признаков значительно превосходят базовые версии. Собственная реализация алгоритма показала результаты, сопоставимые со sklearn, что подтверждает правильность её работы


### Сводная таблица по регрессии:

| Модель                               | MSE       | MAE      | R^2    |
| ------------------------------------ | --------- | -------- | ----- |
| Sklearn (до улучшения)               | 2.316e+08 |  10754   | 0.903 |
| Sklearn (после улучшения)            | 2.406e+08 |   9407   | 0.900 |
| Собственная модель (до улучшения)    | 2.408e+08 |  10642   | 0.900 |
| Собственная модель (после улучшения) | 2.414e+08 |   9374   | 0.899 |

Вывод: все модели показывают близкие результаты. Улучшения дают более стабильные метрики, но радикального роста качества нет

### Сводная таблица по классификации:

| Модель                               | Accuracy  | Precision | Recall | F1-score |
| ------------------------------------ | --------- | --------- | ------ | -------- |
| Sklearn (до улучшения)               |    0.739  |    0.737  | 0.739  | 0.737    |
| Sklearn (после улучшения)            |    0.825  |    0.825  | 0.825  | 0.823    |
| Собственная модель (до улучшения)    |    0.739  |    0.737  | 0.739  | 0.737    |
| Собственная модель (после улучшения) |    0.825  |    0.825  | 0.825  | 0.823    |

Вывод: качественная обработка категориальных признаков и подбор гиперпараметра k значительно улучшают работу классификатора. Улучшенные модели достигают прироста более чем **8%** по accuracy.


